In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

In [2]:
def total_energy(lattice):
    energy = 0
    rows, cols = lattice.shape
    for i in range(rows):
        for j in range(cols):
            spin = lattice[i, j]
            neighbor_sum = (
                lattice[(i + 1) % rows, j] +
                lattice[i, (j + 1) % cols] +
                lattice[(i - 1) % rows, j] +
                lattice[i, (j - 1) % cols]
            )
            energy += -spin * neighbor_sum
            
    return energy

In [3]:
def calculate_energy(lattice, i, j):
    size = lattice.shape[0]
    
    spin_ij = lattice[i, j]

    neighbors_indices = [((i+1) % size, j), ((i-1+size) % size, j), (i, (j+1) % size), (i, (j-1+size) % size)]

    dE = 0

    for ni, nj in neighbors_indices:
        if 0 <= ni < size and 0 <= nj < size:
            spin_neighbor = lattice[ni, nj]
            dE += 2 * spin_ij * spin_neighbor

    return dE

In [4]:
def metropolis_step(lattice, temperature, energy):
    
    size = lattice.shape[0]
    i, j = random.randint(0, size-1), random.randint(0, size-1)
    
    beta= 1/temperature
    while lattice[i,j] == 0:
        i, j = random.randint(0, size-1), random.randint(0, size-1)
        
    dE = calculate_energy(lattice, i, j)
    
    if (dE > 0) and (np.random.random() < np.exp(-dE*beta)):
        lattice[i, j] *= -1
    
    elif dE <= 0:
        lattice[i, j] *= -1
        
    if energy == True:
        return total_energy(lattice)

In [5]:
def swendsen_wang_step(lattice, temperature):
    
    size = lattice.shape[0]
    visited = np.full_like(lattice, -1)

    clusters = []
    
    # Calculate bond probability based on temperature and coupling constant J
    J = 1.0  # Coupling constant (you can adjust this as needed)
    kb = 1.0  # Boltzmann constant
    p = 1.0 - np.exp(-2 * J / (kb * temperature))

    for i in range(size):
        for j in range(size):
            if visited[i, j] == -1:
                cluster = set()
                stack = [(i, j)]

                while stack:
                    x, y = stack.pop()
                    if visited[x, y] == -1:
                        visited[x, y] = 1
                        cluster.add((x, y))
                        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                            nx, ny = (x + dx) % size, (y + dy) % size
                            if visited[nx, ny] == -1 and lattice[x, y] == lattice[nx, ny] and np.random.rand() < p:
                                stack.append((nx, ny))

                clusters.append(cluster)
    
    for cluster in clusters:
        if random.random() < 0.5:
            for i, j in cluster:
                lattice[i, j] *= -1

In [51]:
def combined_metropolis_swendsen_wang(lattice, temperature, m, n):

    magn_= np.zeros(10000)
    
    for _ in tqdm (range(m), desc="Processing Metropolis"):
        metropolis_step(lattice, temperature, False)
    for _ in tqdm (range(n), desc="Processing Swendsen-Wang"):
        swendsen_wang_step(lattice, temperature)
    for _ in tqdm (range(10000), desc="Processing Metropolis2"):
        metropolis_step(lattice, temperature, False)
        magn_[_]= abs(lattice.sum()/np.count_nonzero(lattice))
       
    susc= magn_[-10000:].std()
    magn= magn_[-10000:].mean()
    return magn,susc

In [52]:
sizes= [10,25,50,100,200]
temperature = 2.26
m_steps = 1000000
n_steps = 50
magn_=[]
sus_=[]
for i in range(len(sizes)):
    lattice = np.random.choice([-1, 1], size=(sizes[i], sizes[i]))
    magn, susc= combined_metropolis_swendsen_wang(lattice, temperature, m_steps, n_steps)
    magn_.append(abs(magn))
    sus_.append(susc)

Processing Metropolis2: 100%|██████████| 10000/10000 [00:01<00:00, 6754.58it/s]


In [53]:
x= np.log(sizes).reshape(-1,1)
y= np.log(magn_)
model = LinearRegression()
model.fit(x, y)
coefficients = model.coef_
print("Coefficient:", coefficients[0])

Coefficient: -0.1223131841974869


In [55]:
x= np.log(sizes).reshape(-1,1)
y= np.log(sus_)
model = LinearRegression()
model.fit(x, y)
coefficients = model.coef_
print("Coefficient:", coefficients[0])

Coefficient: -1.7313995633053794


In [49]:
magn_

[0.7405683200000001, 0.76983544, 0.4068946, 0.633598485, 0.68393186875]

In [10]:
sus_

[0.10874719372930962,
 0.1876480570776047,
 0.16737388401914535,
 0.22657333477885808,
 0.2453375982366046]

In [25]:
x= np.log(sizes).reshape(-1,1)
y= np.log(sus_)
model = LinearRegression()
model.fit(x, y)
coefficients = model.coef_
print("Coefficient:", coefficients[0])

Coefficient: -1.7713880444645367
